Assignment 4

Problem 1

In [ ]:

#1.0 preprocessing

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.decomposition import TruncatedSVD
from nltk.stem import PorterStemmer
cv = CountVectorizer()
ps = PorterStemmer()
tv = TfidfVectorizer()
from string import punctuation, digits
import nltk
import re
import pickle
import numpy
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

# imports first, assignments below
cv = CountVectorizer()
ps = PorterStemmer()
tv = TfidfVectorizer()


# put file into dataframe
df = pd.read_csv("news_label.csv", encoding='latin1')
docs = df["Message"] # make a series, not a list, as a test
labels = df["Class"]


# make preprocessing function
def preprocess_text(text):
  text = text.lower() #make everything lower case
  text = text.translate(str.maketrans('','', digits)) # take out digits
  text = text.translate(str.maketrans('','', punctuation)) # take out punctuation
  text = re.sub(' +', ' ', text).strip() # remove extra spaces
  t_list = text.split(" ")
  text = " ".join(ps.stem(word) for word in t_list) #stemming
  return text

mystopwords = list(ENGLISH_STOP_WORDS)
mystopwords.sort()
stem_stopwords = [ps.stem(word) for word in mystopwords]


cv = CountVectorizer(preprocessor=preprocess_text,
                     stop_words = stem_stopwords,
                     ngram_range=(1,1),
                     max_df = docs.shape[0]-1,
                     min_df = 2,
                     max_features = 100)

tv = TfidfVectorizer(preprocessor=preprocess_text,
                      stop_words = stem_stopwords,
                      norm=None,
                      smooth_idf=False,
                      ngram_range = (1,1),
                      max_df=docs.shape[0]-1,
                      max_features = 100)

dtm1 = cv.fit_transform(docs)

# 2.2 Ctf:
terms = cv.get_feature_names_out()
freq_sum = dtm.sum(axis=0)[0].tolist()[0] # list within a list to get 1st element
Ctf = pd.DataFrame({"terms": terms,
                    "freq": freq_sum})
Ctf.sort_values("freq",ascending=False,inplace=True)


# 1.1
dict1 = pd.DataFrame({"terms":terms, "freq":freq_sum})
                  # freq at least 100
myvocab1 = dict1.sort_values("freq",ascending=False,inplace=True)
print("Printing myvocab1:")
print(myvocab1)

dtm11 = tv.fit_transform(docs) # for tv later

# 1.2
print("Printing dtm1:")
print(dtm1)

# load partition
with open("partition_hw4.pkl", "rb") as f:
  partition = pickle.load(f)
  #print(partition)

# convert to discrete labels to binary 0-1 labels
num_labels = labels.replace(["med","space"],[0,1])

# convert sparse matrix to dataframe - sparse or dense matrix, depending
dtm_df = pd.DataFrame(dtm.toarray(),columns=tv.get_feature_names_out())
#print("Printing dtm_df:",dtm_df)

# testing modules - split data into training and testing
train_df = df.iloc[partition["train_inds"]]
#print(train_df)
test_df = df.iloc[partition["test_inds"]]
#print(test_df)

train_text = train_df["Message"].apply(preprocess_text)
test_text =  test_df["Message"].apply(preprocess_text)

# convert discrete labels to binary 0-1 labels
num_labels1 = labels.replace(["med", "space"],[0,1])

# train-test split using indices
# partition data
train_inds = partition["train_inds"] #ndarray
test_inds = partition["test_inds"] #list
X_train = dtm_df.iloc[train_inds] #df
X_test = dtm_df.iloc[test_inds] #df
y_train = np.array(num_labels1)[train_inds] # ndarray
y_test = np.array(num_labels1)[test_inds] # ndarray


# train for KNN classifier
k = 7
kNN = KNeighborsClassifier(n_neighbors=k).fit(X_train, y_train)
test_pred = kNN.predict(X_test)
#print(test_pred)

# 1.3 test the KNN case 
pred_label_knn1 = kNN.predict(X_test)
print("Print pred_label_knn1:")
print(pred_label_knn1)

# 1.4 get accuracy of model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
knn_contable = confusion_matrix(y_test, test_pred, labels=[0,1])
knn_disp = ConfusionMatrixDisplay(confusion_matrix=knn_contable, 
                                  display_labels=[0,1])
accuracy_knn1 = (knn_contable[0,0] + knn_contable[1,1])/np.sum(knn_contable)
accuracy_knn11 = kNN.score(X_test,y_test)
print("Printing accuracy_knn1:")
print(accuracy_knn1)
#print(accuracy_knn11) # yep, it matches exactly ^_^

# 1.5
precision_knn1 = precision_score(y_test, pred_label_knn1, pos_label = 1)
print("Printing precision_knn1:")
print(precision_knn1)

# 1.6
recall_knn1 = recall_score(y_test, pred_label_knn1, pos_label = 1)
print("Printing recall_knn1:")
print(recall_knn1)


/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['anywh', 'becau', 'el', 'elsewh', 'everywh', 'ind', 'otherwi', 'plea', 'somewh'] not in stop_words.
  warnings.warn(


Printing myvocab1:
None
Printing dtm1:
  (0, 99)	3
  (0, 17)	6
  (0, 34)	1
  (0, 90)	1
  (0, 63)	1
  (0, 6)	1
  (0, 14)	1
  (0, 56)	2
  (0, 71)	2
  (0, 52)	1
  (0, 72)	1
  (0, 66)	1
  (0, 35)	1
  (0, 57)	1
  (0, 38)	1
  (0, 32)	1
  (0, 1)	1
  (0, 64)	1
  (0, 2)	1
  (0, 30)	2
  (0, 10)	1
  (0, 87)	1
  (1, 1)	1
  (1, 98)	1
  (1, 88)	2
  :	:
  (1143, 34)	1
  (1143, 52)	1
  (1143, 64)	1
  (1143, 88)	1
  (1143, 40)	1
  (1143, 3)	2
  (1143, 37)	2
  (1143, 89)	1
  (1143, 45)	1
  (1143, 13)	1
  (1143, 81)	3
  (1143, 21)	1
  (1144, 99)	2
  (1144, 1)	1
  (1144, 10)	2
  (1144, 69)	1
  (1144, 45)	1
  (1144, 27)	1
  (1144, 25)	1
  (1144, 95)	1
  (1144, 65)	1
  (1144, 48)	3
  (1144, 47)	2
  (1144, 80)	1
  (1144, 44)	2
Print pred_label_knn1:
[0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 1 1 0 1 1 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 0 1
 0 0 1 1 0 1 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 1 0 0
 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 1 1 1 0 0 0 1 0 1 0 1 1 0
 1 1 1 1 0 1 1 1 0 0 1 0 1 0 0 0 1 0 0

Problem 2

In [ ]:
# Continuation of problem 1:

# Work to impore the kNN model, to get accuracy on the testing set to above 90%
# K = 2 to 20 permutations didn't work and stayed at ~80%
# train for KNN classifier:
k_values = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
results = []
for k in k_values:
  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(X_train, y_train)
  test_pred = kNN.predict(X_test)
  # 1.3 test the KNN case 
  pred_label_knn1 = knn.predict(X_test)
  # 1.4 get accuracy of model
  from sklearn.metrics import confusion_matrix
  from sklearn.metrics import ConfusionMatrixDisplay
  knn_contable = confusion_matrix(y_test, test_pred, labels=[0,1])
  knn_disp = ConfusionMatrixDisplay(confusion_matrix=knn_contable, 
                                  display_labels=[0,1])
  accuracy_knn1 = (knn_contable[0,0] + knn_contable[1,1])/np.sum(knn_contable)
  #accuracy_knn11 = knn.score(X_test,y_test)
  print("Printing accuracy_knn1:")
  print(accuracy_knn1)
  #print(accuracy_knn11) # yep, it matches exactly ^_^
  # 1.5
  precision_knn1 = precision_score(y_test, pred_label_knn1, pos_label = 1)
  print("Printing precision_knn1:")
  print(precision_knn1)
  # 1.6
  recall_knn1 = recall_score(y_test, pred_label_knn1, pos_label = 1)
  print("Printing recall_knn1:")
  print(recall_knn1)

# Work to improve model



NameError: ignored

Problem 3

In [ ]:
# Continuation of problem 2:

# Build neural network models

# 3.1 h_val_auc:

# 3.2 best_model:
